In [1]:
!pip install torch torchvision

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:06<00:00, 28.3MB/s] 


In [9]:
# Define model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [11]:
# Set up the training loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 2000] loss: 2.193
[Epoch 1, Batch 4000] loss: 2.003
[Epoch 1, Batch 6000] loss: 1.797
[Epoch 1, Batch 8000] loss: 1.682
[Epoch 1, Batch 10000] loss: 1.591
[Epoch 1, Batch 12000] loss: 1.538
[Epoch 2, Batch 2000] loss: 1.487
[Epoch 2, Batch 4000] loss: 1.441
[Epoch 2, Batch 6000] loss: 1.410
[Epoch 2, Batch 8000] loss: 1.392
[Epoch 2, Batch 10000] loss: 1.385
[Epoch 2, Batch 12000] loss: 1.341
[Epoch 3, Batch 2000] loss: 1.315
[Epoch 3, Batch 4000] loss: 1.285
[Epoch 3, Batch 6000] loss: 1.271
[Epoch 3, Batch 8000] loss: 1.259
[Epoch 3, Batch 10000] loss: 1.257
[Epoch 3, Batch 12000] loss: 1.239
[Epoch 4, Batch 2000] loss: 1.184
[Epoch 4, Batch 4000] loss: 1.200
[Epoch 4, Batch 6000] loss: 1.183
[Epoch 4, Batch 8000] loss: 1.181
[Epoch 4, Batch 10000] loss: 1.171
[Epoch 4, Batch 12000] loss: 1.165
[Epoch 5, Batch 2000] loss: 1.085
[Epoch 5, Batch 4000] loss: 1.132
[Epoch 5, Batch 6000] loss: 1.107
[Epoch 5, Batch 8000] loss: 1.112
[Epoch 5, Batch 10000] loss: 1.114
[Epoc

In [12]:
# Evaluate the model’s performance
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 59.31 %


In [13]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
№Load the model